In [1]:
import json
import pandas as pd
import os
import pickle
import math
import cv2
from tqdm import tqdm
import numpy as np

Please adapt according to your case:
* img_data_path: specify the path to the images
* out_data_path: specify the path to where the .json file should be saved
* anatation parameters:
    * cam_intrinsic: specify the intrinsic camera parameters (If not correctly specified, the inferred bounding box will mismatch the size of an object)

## Generate the carla test json files

In [2]:
img_data_path = '/home/mmvg/Desktop/mmdetection3d/data/Carla/'
img_data_path = r"D:\A_Monash_Cyc_Near_Miss_Detection\CARLA_Data\Carla_data_unziped"

scenes = os.listdir(img_data_path)
if '.DS_Store' in scenes:
    scenes.remove('.DS_Store')
if 'carla_train.json' in scenes:
    scenes.remove('carla_train.json')
total_anns = []
total_imgs = []
cate = set()
ind = 0
for scene in scenes:
    images = os.listdir(os.path.join(img_data_path,scene,'Images'))
    images = [os.path.join(scene,'Images',x) for x in images]
    with open(os.path.join(img_data_path,scene,'GT/gt.txt'),'r') as f:
        gts = f.readlines()
    for image in images:
        info = {}
        info['file_name'] = image
        image_path_split = os.path.normpath(image).split(os.path.sep)    # Split the path according to different OS
        info['id'] = '_'.join([image_path_split[0],image_path_split[2]])
        info['token'] = '_'.join([image_path_split[0],image_path_split[2]])
        info['cam2ego_rotation'] = [0.4998015430569128, -0.5030316162024876, 0.4997798114386805, -0.49737083824542755]
        info['cam2ego_translation'] = [1.70079118954, 0.0159456324149, 1.51095763913]
        info['ego2global_rotation'] = [0.5720320396729045, -0.0016977771610471074, 0.011798001930183783, -0.8201446642457809]
        info['ego2global_translation'] = [411.3039349319818, 1180.8903791765097, 0.0]
        info['cam_intrinsic'] = [[1266.417203046554, 0.0, 816.2670197447984],[0.0, 1266.417203046554, 491.50706579294757],[0.0, 0.0, 1.0]]
        info['width'] = 1600
        info['height'] = 900
        total_imgs.append(info)
    for gt in gts:
        gt_info = gt[:-1].split(',')
        image_name = "%05d.jpeg" % int(gt_info[0])
        image_name = scene+'/Images/'+image_name
        if math.sqrt(float(gt_info[13])**2+float(gt_info[14])**2) < 30:
            info={}
            info['file_name'] = image_name
            info['image_id'] = '_'.join([image_name.split('/')[0],image_name.split('/')[2]])
            info['area'] = (float(gt_info[8])-float(gt_info[6]))*(float(gt_info[9])-float(gt_info[7]))
            info['bbox'] = [float(gt_info[6]),float(gt_info[7]),float(gt_info[8])-float(gt_info[6]),float(gt_info[9])-float(gt_info[7])]
            if gt_info[2] == ' vehicle':
                info['category_name'] = 'car'
                info['category_id'] = 0
            elif gt_info[2] == ' pedestrian':
                info['category_name'] = 'pedestrian'
                info['category_id'] = 7
            info['iscrowd'] = 0
            if float(gt_info[16]) < -math.pi:
                angle = 2*math.pi+float(gt_info[16])
            elif float(gt_info[16]) > math.pi:
                angle = float(gt_info[16]) - 2*math.pi
            else:
                angle = float(gt_info[16])
            info['bbox_cam3d'] = [float(gt_info[14]),-float(gt_info[15]),float(gt_info[13]),float(gt_info[11]),float(gt_info[10]),float(gt_info[12]),angle]
            info['velo_cam3d'] = [0,0]
            info['center2d'] = [(float(gt_info[8])+float(gt_info[6]))/2, (float(gt_info[9])+float(gt_info[7]))/2, float(gt_info[13])]
            info['attribute_name'] = 'pedestrian.standing'
            info['attribute_id'] = 3
            info['segmentation'] = []
            info['id'] = ind
            ind+=1
            total_anns.append(info)
print('Total annotation: {}, Total image number:{}'.format(len(total_anns), len(total_imgs)))

49067 12841


In [11]:
# Open data for NuScenes data, to borrow categories to save to CARLA data.
# ann = json.load(open(r'C:\Users\Lingheng\Documents\git_repo_2023\mmdetection3d\data\nuscenes\nuscenes_infos_train_mono3d.coco.json'))
# nuscences_obj_categories = ann['categories']
# nuscences_obj_categories

In [8]:
nuscences_obj_categories = [{'id': 0, 'name': 'car'}, {'id': 1, 'name': 'truck'}, 
                            {'id': 2, 'name': 'trailer'}, {'id': 3, 'name': 'bus'}, 
                            {'id': 4, 'name': 'construction_vehicle'}, {'id': 5, 'name': 'bicycle'},
                            {'id': 6, 'name': 'motorcycle'}, {'id': 7, 'name': 'pedestrian'}, 
                            {'id': 8, 'name': 'traffic_cone'}, {'id': 9, 'name': 'barrier'}]

In [4]:
# Save carla data for FCOS3D
carla_info = {}
carla_info['categories'] = nuscences_obj_categories    # Object categories, shared among NuScenes, Carla and Victorian data
carla_info['annotations'] = total_anns
carla_info['images'] = total_imgs

out_data_path = r"C:\Users\Lingheng\Documents\git_repo_2023\mmdetection3d\data\carla"  # dir to save the data
with open(os.path.join(out_data_path, 'carla_train.json'),'w') as f:
    json.dump(carla_info,f)

## Generate the Victorian Cycling Legacy test json files

In [5]:
img_data_path = r"D:\A_Monash_Cyc_Near_Miss_Detection\Cyc_Video_Frame"

participants = os.listdir(img_data_path)
if 'desktop.ini' in participants:
    participants.remove('desktop.ini')
if '.DS_Store' in participants:
    participants.remove('.DS_Store')
if 'carla_train.json' in scenes:
    participants.remove('carla_train.json')

# There is no annotation for Victorian Cycling Legacy dataset
total_imgs = []
for participant in participants:
    # print(participant)
    trips = os.listdir(os.path.join(img_data_path, participant))
    for trip in trips:
        # print('\t'+trip)
        trip_images = os.listdir(os.path.join(img_data_path, participant, trip))
        for trip_img in trip_images:
            image = os.path.join(participant, trip, trip_img)
            info = {}
            info['file_name'] = image
            info['id'] = '_'.join([participant, trip, trip_img])
            info['token'] = '_'.join([participant, trip, trip_img])
            info['cam2ego_rotation'] = [0.4998015430569128, -0.5030316162024876, 0.4997798114386805, -0.49737083824542755]
            info['cam2ego_translation'] = [1.70079118954, 0.0159456324149, 1.51095763913]
            info['ego2global_rotation'] = [0.5720320396729045, -0.0016977771610471074, 0.011798001930183783, -0.8201446642457809]
            info['ego2global_translation'] = [411.3039349319818, 1180.8903791765097, 0.0]
            info['cam_intrinsic'] = [[1266.417203046554, 0.0, 816.2670197447984],[0.0, 1266.417203046554, 491.50706579294757],[0.0, 0.0, 1.0]]
            info['width'] = 1600
            info['height'] = 900
            total_imgs.append(info)
            
print('Total image number:{}'.format(len(total_imgs)))

Total image number:157130


In [6]:
# Save carla data for FCOS3D
vic_cyc_legacy_info = {}
vic_cyc_legacy_info['categories'] = ann['categories']    # Object categories, shared among NuScenes, Carla and Victorian data
vic_cyc_legacy_info['annotations'] = []
vic_cyc_legacy_info['images'] = total_imgs

out_data_path = r"C:\Users\Lingheng\Documents\git_repo_2023\mmdetection3d\data\vic_cyc_legacy"  # dir to save the data
with open(os.path.join(out_data_path, 'vic_cyc_legacy_test.json'),'w') as f:
    json.dump(vic_cyc_legacy_info,f)